In [ ]:
import cv2
from ultralytics import YOLO

# ESP32 camera URL
url = 'http://<ESP32_IP>/stream'

# Load the YOLOv8n model
model = YOLO("yolov8n.pt")

# Open a connection to the video stream
cap = cv2.VideoCapture(url)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture image")
        break

    # Run YOLOv8 inference on the frame
    results = model(frame)

    # Filter results to detect persons only (class ID for person in COCO dataset is 0)
    person_detections = [det for det in results if det.cls == 0]

    # Plot the filtered results
    if person_detections:
        for det in person_detections:
            bbox = det.xyxy  # Bounding box coordinates
            conf = det.conf  # Confidence score
            label = model.names[int(det.cls)]  # Class label
            x1, y1, x2, y2 = map(int, bbox)  # Bounding box coordinates

            # Draw the bounding box and label on the frame
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"{label} {conf:.2f}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Display the result frame
    cv2.imshow('ESP32 Camera YOLOv8 Person Detection', frame)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
